In [1]:
from tensorflow.keras import optimizers, losses, callbacks
from tensorflow.data import TFRecordDataset
from src import models, loader

In [3]:
builder = loader.DatasetBuilder('data/', shape=(256, 256))
builder()

16000it [02:17, 116.69it/s]
4000it [00:12, 326.43it/s]


In [8]:
batch_size = 32

train = TFRecordDataset('data/train.records')
train = train.map(builder.decoder)
train = train.map(builder.data_augmentation)
train = train.shuffle(buffer_size=7000)
train = train.batch(batch_size=batch_size)

In [9]:
test = TFRecordDataset('data/test.records')
test = test.map(builder.decoder)
test = test.map(builder.normalize)
test = test.batch(batch_size=batch_size)

In [10]:
model = models.SqueezeExcitation()
model.build((1, 256, 256, 3))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from datetime import datetime


logdir = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir)
model_checkpoint = ModelCheckpoint('SqueezeExcitation.h5', save_best_only=True)
reduce_lr = ReduceLROnPlateau(factor=0.2, patience=5, min_lr=0.000001)
early_stop = EarlyStopping(patience=10)
_callbacks = [model_checkpoint, reduce_lr, early_stop, tensorboard_callback]

In [ ]:
history = model.fit(train, epochs=100, validation_data=test, callbacks=_callbacks)

Epoch 1/100
500/500 [==============================] - 390s 779ms/step - loss: 4.3633 - accuracy: 0.1098 - val_loss: 12.3983 - val_accuracy: 0.0160 - lr: 0.0010
Epoch 2/100
222/500 [============>.................] - ETA: 3:36 - loss: 3.1659 - accuracy: 0.2869